In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import os
os.chdir("/content/drive/My Drive/DATASCIENCE/Projet")

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import os, sys, datetime, re
from functools import partial

from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer,LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report

from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfTransformer

import matplotlib.pyplot as plt

# New Text Preprocessing/Cleaning Function

In [8]:
pip install unidecode


     |████████████████████████████████| 245kB 3.5MB/s 


In [9]:
# Text pre-processing modules
from bs4 import BeautifulSoup
import unidecode
import spacy, en_core_web_sm
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner']) 
from nltk.corpus import stopwords 
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
STOPWORDS = set(stopwords.words('english')) 

# Compile regular expression
SPEC_CHARS_REPLACE_BY_SPACE = re.compile('[/(){}\[\]\|@,;]')
SPEC_CHARS = re.compile(r'[^a-zA-z0-9\s]')
SPEC_CHARS_INCLUDE_DIGITS = re.compile(r'[^a-zA-z\s]')
EXTRA_NEWLINES = re.compile(r'[\r|\n|\r\n]+')


## Functions for text preprocessing, cleaning

def strip_htmltags(text):
    soup = BeautifulSoup(text,"lxml")
    return soup.get_text()

def replace_accented_chars(text):
    return unidecode.unidecode(text)

def stem_text(text):
    ps = PorterStemmer()
    modified_txt = ' '.join([ps.stem(word) for word in text.split()])
    return modified_txt    

def lemmatize(text):
    modified_text = nlp(text)
    return ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in modified_text])

def normalize(text, method='stem'):
    """ Text normalization to generate the root form of the inflected words.
        This is done by either "stem" or "lemmatize" the text as defined by the 'method' arguments.  
        Note that using "lemmatize" will take much longer to run compared to "stem".
    """
    if method == 'stem':
        return stem_text(text)
    if method == 'lemmatize':
        return lemmatize(text)
    print('Please choose either "stem" or "lemmatize" method to normalize.')
    return text

def rm_special_chars(text, rm_digits=False):
    # remove & replace below special chars with space
    modified_txt = SPEC_CHARS_REPLACE_BY_SPACE.sub(' ', text)
    
    # remove rest of special chars, no replacing with space
    if rm_digits:
        return SPEC_CHARS_INCLUDE_DIGITS.sub('', modified_txt)
    else:
        return SPEC_CHARS.sub('', modified_txt)

def rm_extra_newlines_and_whitespace(text):
    # rm extra newlines
    modified_txt =  EXTRA_NEWLINES.sub(' ', text)
    
    # rm extra whitespaces
    return re.sub(r'\s+', ' ', modified_txt)

def rm_stopwords(text, simple=True):
    """ Remove stopwords using either the simple model with replacement.
        or using nltk.tokenize to split the words and replace each words. This will incur speed penalty.
    """
    if simple:
        return ' '.join(word for word in text.split() if word not in STOPWORDS)
    else:
        tokens = word_tokenize(text)
        tokens = [token.strip() for token in tokens]
        return ' '.join(word for word in tokens if word not in STOPWORDS)


def clean_text(raw_text, strip_html = True, replace_accented = True,
                normalize_text = True, normalize_methd = 'stem',
                remove_special_chars = True, remove_digits = True,
                remove_stopwords = True, rm_stopwords_simple_mode = True):

    """ The combined function for all the various preprocessing method. 
        Keyword args:
            strip_html               : Remove html tags.
            replace_accented         : Convert accented characters to closest English characters.
            normalize_text           : Normalize text based on normalize_methd.
            normalize_methd          : "stem" or "lemmatize". Default "stem".
            remove_special_chars     : Remove special chars.
            remove_digits            : Remove digits/numeric as special characters.
            remove_stopwords         : Stopwords removal basedon NLTK corpus.
            rm_stopwords_simple_mode : skip tokenize before stopword removal. Speed up time.
    """
    
    text = raw_text.lower()
    
    if strip_html:
        text = strip_htmltags(text)
    if replace_accented:
        text = replace_accented_chars(text)
    if remove_special_chars:
        text = rm_special_chars(text, remove_digits)    
    if normalize_text:
        text = normalize(text, normalize_methd)
    if remove_stopwords:
        text = rm_stopwords(text, rm_stopwords_simple_mode)
        
    text = rm_extra_newlines_and_whitespace(text)  
    
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Import Data and basic processing on dataset

In [0]:
fname = 'commerce.csv'
df = pd.read_csv(fname, header=0,sep=';')

# replace nan with best value based on exploration
df['Name'] = df['Name'].fillna(df['Name'].mode()[0])


# Convert attributes to integer
for col in [ 'Name']:
    df[col] = df[col].fillna(-1)
    #df[col] = df[col].astype('int')
    df[col] = df[col].astype('str')
    df[col] = df[col].replace('-1', np.nan)

# Apply the new clean text function
clean_text_p = partial(clean_text, strip_html = False, remove_digits = False,
                       normalize_text = False, normalize_methd = 'stem')

df['title1'] = df['Name'].apply(clean_text_p)

In [11]:
df['title1'].unique()

array(['lg smart tv led 55 4k recepteur integre',
       'televiseur telefunken e700a 49 smart full hd led wifi',
       'lg tv led 49', 'televiseur samsung k6000 43 full hd smart',
       'televiseur samsung smart full hd 40 serie 5 wifi',
       'televiseur maxwell led 55 curved smart 4k wifi gold',
       'condor televiseur full hd 40 bleu',
       'condor televiseur hd 32 1 iptv',
       'televiseur samsung 49 full hd k5100 series 5',
       'televiseur samsung smart full hd 43 serie 5 wifi',
       'samsung 48 full hd smart tv', 'televiseur condor full hd 50 gold',
       'televiseur maxwell vega led 55 smart ultra hd 4k wifi silver',
       'televiseur telefunken e2000 32 led hd',
       'imprimante 4en1 epson workeforce pro wf5620dwf couleur wifi',
       'imprimante couleur reservoir integre a3 epson l1300',
       'imprimante matricielle epson lx350',
       'imprimante photo epson l810 reservoir integre couleur',
       'imprimante matricielle epson lq350',
       'imprimante

# Pipelines, GridSearch and Attributes Classification

In [0]:
# Prepare model -- Drop na and keep those with values
def get_X_Y_data(x_col, y_col):
    sub_df =  df[[x_col, y_col]]
    sub_df.head()
    sub_df = sub_df.dropna()
    return sub_df[x_col], sub_df[y_col]

In [28]:
text_vect_pipe = Pipeline([
                        ('vect', CountVectorizer()),
                        ('tfidf', TfidfTransformer())
                        ])

pred_model = Pipeline([
            ('process', text_vect_pipe),
            ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None))
            ])

parameters = {}
parameters['process__vect__ngram_range'] = [(0,1),(1,2),(1,3)]
parameters['clf__loss'] = ["hinge"]
parameters['clf__alpha'] = [5e-6,1e-5]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

CV = GridSearchCV(pred_model, parameters)
CV.fit(X_train, y_train)
y_pred = CV.predict(X_test)
    
print('accuracy %s' % accuracy_score(y_pred, y_test))
print("=="*18)
print()
print("Details of GridSearch")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


accuracy 0.9571544058205336

Details of GridSearch


#**Faisons une prediction**

In [31]:
X_test

11622    lenovo pc portable ideapad 32015ikbn i5 7e gen...
2347                     pc de bureau dell optiplex 7020mt
9202     lenovo pc portable ideapad 32015ikbra i5 8e ge...
4953     lenovo pc portable ideapad 32015ikbra i5 8e ge...
7528     lenovo pc portable ideapad 32015ikbn i5 7e gen...
                               ...                        
6270     lenovo pc portable ideapad 32015ikbn i5 7e gen...
2650     pc de bureau dell vostro 3667 i3 6e gen 4 go 5...
9892           pc portable lenovo 32015ast bleu e29000 8go
3024     hp pc de bureau prodesk 400 g3 i5 6e gen 4 go ...
12246    lenovo pc portable ideapad 32015ikbra i5 8e ge...
Name: title1, Length: 3711, dtype: object

In [0]:
new=CV.predict(['pc de bureau dell optiplex 7020mt'])

In [33]:
new

array(['PC de bureau\xa0DELL Optiplex 7020MT Intel® Core™ i3-4160 (3.1GHz,3M)\xa0/4GB (1x4GB)1600MHz DDR3 up to 8GB / 500GB SATA III(7200Rpm)\xa0/Integrated Intel® HD Graphics 4600 /2 external USB 3.0 ports ;6 external USB 2.0 ports 1 VGA/\xa0Graveur DVD 16X DVD+/-RW Drive\xa0/free Dos/\xa0Garantie 3 ans'],
      dtype='<U398')